# Pandas assignment
In this assignment, you will complete some functions what will be used in the transversal assignemnt.
You will implement some key functions in the ETL (Extract, Transform, Load) process used to prepare the data for the assignment.
 
First, you will define a function the data in the CSV file "datasets/Online Retail_2.csv" into a dataframe. The function must load only the columns:
- 'InvoiceNo': An invoice number 
- 'StockCode': The stock code of the product
- 'Description': A description of the product
- 'Quantity': Quantity in invoice
- 'InvoiceDate': Date of invoice 
- 'UnitPrice': Unitary price of product 
- 'CustomerID': Identifier of the customer 
- 'Country': Country of customer

In [1]:
import pandas as pd
import numpy as np
from IPython import display
path_to_csv = 'Online Retail_2.csv'

def read_file():
    """
    Reads the e-commerce dataset in Datasets/Online Retail.csv and loads it into a data frame.

    Args:
        None

    Returns: Pandas dataframe containing data
    """
    # Write your code here

    return pd.read_csv(path_to_csv, usecols=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'])

ecommerce_data = read_file()
ecommerce_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Next, you will filter out rows that do not correspond to actual invoices, but returns or purchases. To do so, you will build a **query** that keeps only values where the column quantity is greater than zero.


In [2]:
def get_demand(ec):
    """
    Reads the quantity and returns indexes where the value is greater than 0

    Args:
        ec - A Pandas Dataframe containing e-commerce records.

    Returns: a dataframe with the production site identifier
    """
    # Write your code here
    return ec.query('Quantity > 0')

ecommerce_data = get_demand(ecommerce_data)
ecommerce_data.describe()

,Quantity,UnitPrice,CustomerID
count,531285.000000,531285.000000,397924.000000
mean,10.655262,3.857296,15294.315171
std,156.830323,41.810047,1713.169877
min,1.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15159.000000
75%,10.000000,4.130000,16795.000000
max,80995.000000,13541.330000,18287.000000


Now, we are going to create a product code, just by keeping the first number of the stock code. 
You need to create a new column named 'product_type' with the first character of the column 'StockCode'.
You will then filter any stock code that does not start by a number using the str function **contains**.

In [3]:
def get_product_type(ec):
    """
    Reads the Stock Code and returns the first character of the string.
    This first character is used to identify the product type

    Args:
        ec - A Pandas Dataframe containing e-commerce records.

    Returns: a dataframe with the production site identifier
    """
    # Add a new column labeled 'Product type with the first character of the stock code
    ec['product_type'] = ec['StockCode'].str[0]

    # Filter values that do not start with a digit
    # Write your code here

    # 1st Option, was slower than doing a series.str.contains with regex.
    # return ec[pd.to_numeric(ec.product_type, errors='coerce').notnull()]
    return ec[ec['product_type'].str.contains(r'[0-9]', regex=True)]


ecommerce_data = get_product_type(ecommerce_data)
ecommerce_data.describe()

,Quantity,UnitPrice,CustomerID
count,528886.000000,528886.000000,396370.000000
mean,10.673415,3.258152,15301.602071
std,157.128259,4.375777,1709.982593
min,1.000000,0.000000,12346.000000
25%,1.000000,1.250000,13975.000000
50%,3.000000,2.080000,15159.000000
75%,11.000000,4.130000,16803.000000
max,80995.000000,649.500000,18287.000000


Now that our data is ready, we will answer some questions.
First, add a new column named 'revenue' as the product of the quantity and the unit price.

In [4]:
ecommerce_data["Revenue"] = ecommerce_data["Quantity"] * ecommerce_data["UnitPrice"]
    # Write your code here
ecommerce_data.describe()

,Quantity,UnitPrice,CustomerID,Revenue
count,528886.000000,528886.000000,396370.000000,528886.000000
mean,10.673415,3.258152,15301.602071,19.420129
std,157.128259,4.375777,1709.982593,268.320619
min,1.000000,0.000000,12346.000000,0.000000
25%,1.000000,1.250000,13975.000000,3.750000
50%,3.000000,2.080000,15159.000000,9.900000
75%,11.000000,4.130000,16803.000000,17.400000
max,80995.000000,649.500000,18287.000000,168469.600000


Now, answer the following questions using Pandas. 
What is the product type that generates the highest revenue?
Create a pivot table with index  'Country', column 'product' that shows the accumulated revenue per country and product type


In [8]:
# Write your code here
pd.pivot_table(ecommerce_data,index=["Country"], columns=["product_type"], values=["Revenue"], aggfunc=[np.sum])


sum                                            \
                        Revenue                                             
product_type                  1           2         3          4        5   
Country                                                                     
Australia                596.35   130955.77    311.40    2712.80      NaN   
Austria                  593.76     7257.64    117.38     283.95      NaN   
Bahrain                     NaN      402.90     49.20        NaN      NaN   
Belgium                  191.86    33618.66    195.27     373.30      NaN   
Brazil                    53.55     1069.65       NaN        NaN      NaN   
Canada                   127.00     1946.49    584.37      18.72      NaN   
Channel Islands           10.50    16685.95     40.56     932.55      NaN   
Cyprus                   462.27    10654.61    283.35      92.60     4.68   
Czech Republic              NaN      562.10       NaN      56.88      NaN   
Denmark                   96.06    16349.65    154.56     126.90     9.36   
EIRE                    2579.78   235630.56   2005.56    5109.53   129.24   
European Community          NaN      999.85       NaN        NaN      NaN   
Finland                  220.05    10936.35    252.39     559.32      NaN   
France                  2474.60   166033.56    394.78    2725.77    39.12   
Germany                 4551.53   180535.12   1839.00    3484.38    88.44   
Greece                     3.36     3452.26     34.80     690.60      NaN   
Hong Kong                  5.88     8326.91     15.00     103.10      NaN   
Iceland                   16.80     3399.35       NaN      88.50    10.20   
Israel                   113.91     7609.20       NaN     152.55      NaN   
Italy                     73.08    13732.51    132.48     339.72   305.28   
Japan                      0.85    34315.67       NaN    1103.90      NaN   
Lebanon                     NaN     1368.68     12.60      69.75      NaN   
Lithuania                142.80     1439.06       NaN        NaN      NaN   
Malta                       NaN     1494.07     84.28        NaN      NaN   
Netherlands               86.36   262097.46   1181.28    1262.19      NaN   
Norway                   608.98    26482.17    282.00     344.14      NaN   
Poland                   254.05     5580.13    536.82      65.70      NaN   
Portugal                 123.76    23564.54    640.44     375.60    10.35   
RSA                         NaN      862.56       NaN      68.25      NaN   
Saudi Arabia                NaN      145.92       NaN        NaN      NaN   
Singapore                   NaN     7937.39    107.28     390.48      NaN   
Spain                    205.81    39051.98    693.05     914.37    39.12   
Sweden                   603.06    32438.64      9.36     263.70   102.96   
Switzerland              393.37    45555.82    463.11    1459.83      NaN   
USA                       10.50     3137.36     87.43      91.15      NaN   
United Arab Emirates        NaN     1527.88       NaN      81.30      NaN   
United Kingdom        112025.35  6972819.48  55932.72  322200.28  5652.74   
Unspecified               80.12     4088.53      3.15     168.09      NaN   

                                                                
                                                                
product_type                6          7           8         9  
Country                                                         
Australia                 NaN     279.02     3315.97       NaN  
Austria                   NaN      32.01      442.94     15.00  
Bahrain                   NaN     282.24       19.80       NaN  
Belgium                   NaN     196.81     2231.29    120.15  
Brazil                    NaN        NaN       20.40       NaN  
Canada                    NaN      39.66      399.20       NaN  
Channel Islands           NaN      20.40     2317.48       NaN  
Cyprus                    NaN     129.92     1627.26       NaN  
Czech Republi